# Using RiskGPT in Google Colab

This notebook demonstrates how to use RiskGPT in Google Colab by installing it directly from the GitHub repository and setting up the OpenAI API key.


## 1. Install RiskGPT from GitHub

Since RiskGPT is not available on PyPI, we'll install it directly from the GitHub repository.


In [ ]:
# Install Python 3.12 in Colab (if needed)
!apt-get update
!apt-get install python3.12 python3.12-dev python3.12-distutils -y
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 1

# Verify Python version
!python3 --version


In [ ]:
# Install pip for Python 3.12
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.12

# Install RiskGPT directly from GitHub
!pip install git+https://github.com/thwolter/riskgpt.git

# Alternative: If you have access to the wheel file
# !pip install /path/to/dist/riskgpt-0.1.0-py3-none-any.whl


## 2. Set up the OpenAI API Key

You need to set your OpenAI API key to use RiskGPT. In Colab, you can set environment variables directly in the notebook.


In [ ]:
import os
from getpass import getpass

# Prompt for OpenAI API key (this way it won't be visible in the notebook)
openai_api_key = getpass("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

# Optional: Set other configuration variables if needed
os.environ["OPENAI_MODEL_NAME"] = "openai:gpt-4.1-nano"  # Default model
os.environ["MEMORY_TYPE"] = "buffer"  # Default memory type


## 3. Basic Usage Example

Here's a simple example of how to use RiskGPT for risk analysis.


In [ ]:
# Import required modules
from riskgpt import configure_logging
from riskgpt.models.common import BusinessContext
from riskgpt.workflows.risk_workflow import run_risk_workflow

# Configure logging
configure_logging()

# Create a business context
context = BusinessContext(
    project_id="ACME-1",
    project_name="ACME Corp Security Upgrade",
    description="Implement new cybersecurity measures across all departments"
)

# Run the risk workflow
result = run_risk_workflow(context)

# Display the results
print(f"Identified Risks: {len(result.risks)}")
for i, risk in enumerate(result.risks, 1):
    print(f"\nRisk {i}: {risk.title}")
    print(f"Description: {risk.description}")
    print(f"Impact: {risk.impact}")
    print(f"Probability: {risk.probability}")
    print(f"Mitigation: {risk.mitigation}")


## 4. Advanced Usage: External Context Enrichment

RiskGPT can enrich your analysis with external context from web searches.


In [ ]:
from riskgpt.api import search_context

# Search for recent news about cybersecurity
results, ok = search_context("ACME Corp cybersecurity", "news")

if ok:
    print(f"Found {len(results)} search results:")
    for i, result in enumerate(results, 1):
        print(f"\nResult {i}: {result.title}")
        print(f"URL: {result.url}")
        print(f"Snippet: {result.snippet[:100]}...")
else:
    print("Search failed or returned no results.")


## 5. Troubleshooting

If you encounter any issues, here are some common troubleshooting steps:

1. **Python Version**: RiskGPT requires Python 3.12 or higher. Make sure you've installed and are using the correct Python version.

2. **API Key**: Ensure your OpenAI API key is valid and has sufficient credits.

3. **Dependencies**: If you encounter dependency issues, you can try installing them manually:

```python
!pip install langchain>=0.3.26 langchain-openai>=0.3.25 langgraph>=0.4.8 pydantic>=2.11.7
```

4. **Import Errors**: If you see import errors, make sure the package was installed correctly:

```python
!pip list | grep riskgpt
```
